In [1]:
import os
import pywt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchaudio
import numpy as np
import random
from pytorch_lightning.loggers import WandbLogger
import wandb
import auraloss
import collections
from tqdm import tqdm
import pretty_midi
import matplotlib.pyplot as plt
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB
print(torch.cuda.is_available())
import plotly.graph_objects as go
from torch.optim import lr_scheduler
from IPython.display import Audio
from torchaudio.transforms import Fade
import musdb
import museval
import gc
import pandas as pd
import sklearn
from sklearn.metrics import recall_score, precision_score, f1_score, jaccard_score, accuracy_score,zero_one_loss

True


In [2]:
class Track:
    def __init__(self, name, midi_path, drum_path, mix_path):
        self.name = name
        self.midi_path = midi_path
        self.drum_path = drum_path
        self.mix_path = mix_path
        self.targets = {'drums': '', 'bass': ''}
        self.rate = 44100
        self.subset = 'test'

In [3]:
class AudioData:
    def __init__(self, audio):
        self.audio = audio

In [4]:
seed_value = 3407
torch.manual_seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
torch.cuda.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
torch.set_float32_matmul_precision('high')

In [5]:
mix_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/audio/full_mix/'
mixes = os.listdir(mix_folder)
mixes = [mix_folder + m for m in mixes]

drum_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/audio/drum_only/'
drum = os.listdir(drum_folder)
drum = [drum_folder + d for d in drum]

beats_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/beats/'
beats = os.listdir(beats_folder)
beats = [beats_folder + b for b in beats]#

class_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/subclass/'
classes = os.listdir(class_folder)
classes = [class_folder + c for c in classes]

midi_folder = 'D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/midi/'
midis = os.listdir(midi_folder)
midis = [midi_folder + m for m in midis]

In [6]:
all_tracks = []
for idx, val in tqdm(enumerate(classes)):

    name = val.replace('D:/Github/phd-drum-sep/data/MDBDrums-master/MDB Drums/annotations/subclass/', '')
    name = name.replace('_subclass.txt', '')

    t = Track(name, midis[idx], drum[idx], mixes[idx])
    all_tracks.append(t)

23it [00:00, ?it/s]


In [7]:
def turn_transcription_into_roll(transcription, frames):
    # Determine your sampling frequency (frames per second)
    fs = 44100
    
    piano_roll_length = int(frames)
    
    # Initialize the piano roll array
    piano_roll = np.zeros((64, piano_roll_length))
    
    # Fill in the piano roll array
    for note in transcription.instruments[0].notes:
        # Convert start and end times to frame indices
        start_frame = int(np.floor(note.start * fs))
        end_frame = int(np.ceil(note.end * fs))
        
        # Set the corresponding frames to 1 (or note.velocity for a velocity-sensitive representation)
        piano_roll[note.pitch, start_frame:end_frame] = 1  # Or use note.velocity
        
    roll = np.vstack([piano_roll[35:36, :], piano_roll[38:39, :], piano_roll[42:43, :], piano_roll[47:48, :], piano_roll[49:50, :]])
    return roll

In [8]:
def expand(x, out_size=44100*4, step=4410):
    output_tensor = torch.zeros((5, out_size))
    for i in range(x.shape[1]):  # Iterate over the second dimension
        start_idx = i * step
        end_idx = start_idx + step
        output_tensor[:, start_idx:end_idx] = x[:, i].unsqueeze(1)
    return output_tensor

def compress(x, original_shape=(5, 40), step=4410):
    """
    Compresses a tensor from a larger size to its original smaller size by averaging blocks of values.
    
    Args:
    - x (Tensor): The input tensor to be compressed, expected to have the shape (5, 44100) or similar.
    - original_shape (tuple): The shape of the output tensor, default is (5, 40).
    - step (int): The size of the block to average over, default is 4410.
    
    Returns:
    - Tensor: The compressed tensor with shape specified by `original_shape`.
    """
    output_tensor = torch.zeros(original_shape)
    for i in range(original_shape[1]):  # Iterate over the second dimension of the target shape
        start_idx = i * step
        end_idx = start_idx + step
        # Take the mean of each block and assign it to the corresponding position in the output tensor
        output_tensor[:, i] = x[:, start_idx:end_idx].mean(dim=1)
    return output_tensor


In [9]:
def load_audio(path):
    audio_tensors = []
    waveform, _ = torchaudio.load(path)
    return waveform

def load_roll(path, frames):
    transcription = pretty_midi.PrettyMIDI(path)
    roll = turn_transcription_into_roll(transcription, frames)

    return torch.from_numpy(roll).float()


In [ ]:
models = ['ht_epoch_280', 'trans_epoch_280', 'conv_epoch_200']

for m in models:
    out_dir = f"D:/Github/phd-drum-sep/model-as-adt/results_{m}/adt/"
    rows = []
    for track in tqdm(all_tracks):
    
        mixture_tensor = load_audio(track.mix_path)
        shape = mixture_tensor.shape[1]
        
        snippet_length = (mixture_tensor.shape[1] // (44100 * 4)) * (44100 * 4)
        mixture_tensor = mixture_tensor[:, :snippet_length]
    
        roll_tensor = load_roll(track.midi_path, shape)
        roll_tensor = roll_tensor[:, :snippet_length]
    
        proposed_answers = []
    
        track_dir = f'{out_dir}{track.name}'
        track_folder = os.listdir(track_dir)
        segments = len(track_folder) - 2
    
        chunk_len = int(44100 * 4)
    
        for i in range(segments):
            proposed_transcription = pd.read_csv(f'{track_dir}/{i}.csv')
            proposed_transcription = np.asarray(proposed_transcription)
            proposed_transcription = np.asarray(proposed_transcription.T)[1:].T
            proposed_transcription = expand(torch.from_numpy(proposed_transcription))
    
            start = i * chunk_len
            end = start + chunk_len
    
            drum_chunk_ = roll_tensor[:, start:end].numpy()
            proposed_transcription_ = proposed_transcription.numpy()
    
            for drum in range(5):
                drum_chunk = drum_chunk_[drum, :]
                proposed_transcription = proposed_transcription_[drum, :]
                recall = recall_score(drum_chunk, proposed_transcription, average='weighted', zero_division=0)
                precision = precision_score(drum_chunk, proposed_transcription, average='weighted', zero_division=0)
                f1 = f1_score(drum_chunk, proposed_transcription, average='weighted')
                jaccard = jaccard_score(drum_chunk, proposed_transcription, average='weighted')
                acc = accuracy_score(drum_chunk, proposed_transcription)
                rows.append([track.name, i, drum, recall, precision, f1, jaccard, acc])

            drum_chunk = drum_chunk_.flatten()
            proposed_transcription = proposed_transcription_.flatten()
            recall = recall_score(drum_chunk, proposed_transcription, average='weighted', zero_division=0)
            precision = precision_score(drum_chunk, proposed_transcription, average='weighted', zero_division=0)
            f1 = f1_score(drum_chunk, proposed_transcription, average='weighted')
            jaccard = jaccard_score(drum_chunk, proposed_transcription, average='weighted')
            acc = accuracy_score(drum_chunk, proposed_transcription)
            rows.append([track.name, i, 'all', recall, precision, f1, jaccard, acc])
                
    df_results = pd.DataFrame(rows, columns=['track_name', 'slice', 'drum', 'recall', 'precision', 'f1', 'jaccard', 'accuracy', 'zero_one'])
    df_results.to_csv(f"D:/Github/phd-drum-sep/model-as-adt/results_final/{m}.csv")
    

    

  0%|                                                                                                                                                                                                                | 0/23 [00:00<?, ?it/s]

['MusicDelta_80sRock', 0, 0, 0.7999546485260771, 0.6399274396984795, 0.7110484036056073, 0.6399274396984795, 0.7999546485260771]
['MusicDelta_80sRock', 0, 1, 0.8999773242630386, 0.8099591841876584, 0.8525987903585374, 0.8099591841876584, 0.8999773242630386]
['MusicDelta_80sRock', 0, 2, 1.0, 1.0, 1.0, 1.0, 1.0]
['MusicDelta_80sRock', 0, 3, 1.0, 1.0, 1.0, 1.0, 1.0]
['MusicDelta_80sRock', 0, 4, 0.975, 1.0, 0.9873417721518987, 0.975, 0.975]
['MusicDelta_80sRock', 0, 'all', 0.9349863945578232, 0.8832909447045492, 0.9084037928668373, 0.8788744899810265, 0.9349863945578232]
['MusicDelta_80sRock', 1, 0, 0.6704365079365079, 0.6878159802162551, 0.6788763994858453, 0.5580608137405415, 0.6704365079365079]
['MusicDelta_80sRock', 1, 1, 0.7545294784580499, 0.8443977703861434, 0.7965995756320471, 0.6980979455720577, 0.7545294784580499]
['MusicDelta_80sRock', 1, 2, 0.85, 1.0, 0.9189189189189189, 0.85, 0.85]
['MusicDelta_80sRock', 1, 3, 0.95, 1.0, 0.9743589743589742, 0.95, 0.95]
['MusicDelta_80sRock', 1

  4%|████████▋                                                                                                                                                                                               | 1/23 [00:14<05:24, 14.77s/it]

['MusicDelta_80sRock', 8, 'all', 0.8820385487528345, 0.9006816623030499, 0.8912350662915878, 0.8382424411516513, 0.8820385487528345]
['MusicDelta_Beatles', 0, 0, 0.6749773242630386, 0.7382373164023736, 0.7051914608835057, 0.5905898531218988, 0.6749773242630386]
['MusicDelta_Beatles', 0, 1, 0.7045351473922903, 0.8560242866177893, 0.7599240777124928, 0.6344343893771285, 0.7045351473922903]
['MusicDelta_Beatles', 0, 2, 0.825, 1.0, 0.9041095890410958, 0.825, 0.825]
['MusicDelta_Beatles', 0, 3, 0.8, 1.0, 0.8888888888888888, 0.8, 0.8]
['MusicDelta_Beatles', 0, 4, 0.825, 1.0, 0.9041095890410958, 0.825, 0.825]
['MusicDelta_Beatles', 0, 'all', 0.7659024943310657, 0.9154621888583266, 0.8305659692803352, 0.7309920769326893, 0.7659024943310657]
['MusicDelta_Beatles', 1, 0, 0.7499829931972789, 0.7499713222023504, 0.7499771575469201, 0.6456608761738056, 0.7499829931972789]
['MusicDelta_Beatles', 1, 1, 0.8465702947845805, 0.8643000238363351, 0.8547791154999805, 0.7768608219987595, 0.8465702947845805]

  9%|█████████████████▍                                                                                                                                                                                      | 2/23 [00:29<05:08, 14.67s/it]

['MusicDelta_Beatles', 8, 'all', 0.8739739229024943, 0.9552796067457542, 0.9107116044611742, 0.8517603263693478, 0.8739739229024943]
['MusicDelta_BebopJazz', 0, 0, 0.8954535147392291, 0.9325633896541735, 0.9114601825578553, 0.8572926172784533, 0.8954535147392291]
['MusicDelta_BebopJazz', 0, 1, 0.574937641723356, 0.5478911662107524, 0.4902012207141671, 0.36242642246419265, 0.574937641723356]
['MusicDelta_BebopJazz', 0, 2, 0.9499943310657596, 0.9499888075217683, 0.949991569285735, 0.926239087333724, 0.9499943310657596]
['MusicDelta_BebopJazz', 0, 3, 1.0, 1.0, 1.0, 1.0, 1.0]
['MusicDelta_BebopJazz', 0, 4, 0.95, 1.0, 0.9743589743589742, 0.95, 0.95]
['MusicDelta_BebopJazz', 0, 'all', 0.874077097505669, 0.8450380181047917, 0.8573290169808677, 0.7953387648770099, 0.874077097505669]
['MusicDelta_BebopJazz', 1, 0, 0.9704591836734694, 0.9840300703505335, 0.9751893840464148, 0.9563966183261429, 0.9704591836734694]
['MusicDelta_BebopJazz', 1, 1, 0.628344671201814, 0.6620970156770192, 0.57522571673

 13%|██████████████████████████                                                                                                                                                                              | 3/23 [01:09<08:48, 26.40s/it]

['MusicDelta_BebopJazz', 24, 'all', 0.8861269841269841, 0.882247434442118, 0.8841657608023564, 0.8289016829818595, 0.8861269841269841]
['MusicDelta_Britpop', 0, 0, 0.8999773242630386, 0.8099591841876584, 0.8525987903585374, 0.8099591841876584, 0.8999773242630386]
['MusicDelta_Britpop', 0, 1, 0.7249433106575963, 0.5255428036671962, 0.6093450149000489, 0.5255428036671962, 0.7249433106575963]
['MusicDelta_Britpop', 0, 2, 0.7499489795918367, 0.5624234719908372, 0.6427884224624862, 0.5624234719908372, 0.7499489795918367]
['MusicDelta_Britpop', 0, 3, 1.0, 1.0, 1.0, 1.0, 1.0]
['MusicDelta_Britpop', 0, 4, 0.975, 1.0, 0.9873417721518987, 0.975, 0.975]
['MusicDelta_Britpop', 0, 'all', 0.8699739229024943, 0.7650296443667012, 0.814133808843022, 0.7612044961448676, 0.8699739229024943]
['MusicDelta_Britpop', 1, 0, 0.7852154195011338, 0.7851953694554163, 0.7852053936331279, 0.6841227076067407, 0.7852154195011338]
['MusicDelta_Britpop', 1, 1, 0.7397448979591836, 0.6814902355384216, 0.6832677051500204,